In [1]:
## This file implements neural networks and random forest on p002ypresabsSTCC_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 95.00% for over-sampling data and 93.5% after regularization.
## For random forest, the accuracy is 95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 96.56%.

In [14]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [15]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p002ypresabsSTCC_qual.csv')
df.shape

(255, 1761)

In [16]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [17]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     1
12     0
13     0
14     0
15     0
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [18]:
df['pheno'].value_counts()

0    222
1     30
2      3
Name: pheno, dtype: int64

In [19]:
df.head()

,id,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,5,5,0


In [20]:
df_clean = df.drop(columns=['id'])

In [21]:
df_clean.shape

(255, 1760)

In [22]:
df_clean.head()

,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,...,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,5,5,0
4,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,5,5,0


In [23]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1759) (255,)


In [26]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 222), (1, 222), (2, 222)]


In [27]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [28]:
############# Fully-Connected Neural Network ################

In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [30]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [31]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 401us/step - loss: 1.3846 - accuracy: 0.6502 - val_loss: 0.7466 - val_accuracy: 0.6450
Epoch 2/100
466/466 [==============================] - 0s 110us/step - loss: 0.5617 - accuracy: 0.7575 - val_loss: 0.4731 - val_accuracy: 0.7950
Epoch 3/100
466/466 [==============================] - 0s 100us/step - loss: 0.4184 - accuracy: 0.8262 - val_loss: 0.4667 - val_accuracy: 0.8050
Epoch 4/100
466/466 [==============================] - 0s 108us/step - loss: 0.3835 - accuracy: 0.8519 - val_loss: 0.4340 - val_accuracy: 0.8050
Epoch 5/100
466/466 [==============================] - 0s 102us/step - loss: 0.3080 - accuracy: 0.8627 - val_loss: 0.3929 - val_accuracy: 0.8400
Epoch 6/100
466/466 [==============================] - 0s 96us/step - loss: 0.2985 - accuracy: 0.8820 - val_loss: 0.3336 - val_accuracy: 0.8850
Epoch 7/100
466/466 [==============================] - 0s 120us/step - loss: 0.2793 -

Epoch 57/100
466/466 [==============================] - 0s 102us/step - loss: 0.0417 - accuracy: 0.9914 - val_loss: 0.1734 - val_accuracy: 0.9400
Epoch 58/100
466/466 [==============================] - 0s 88us/step - loss: 0.0371 - accuracy: 0.9957 - val_loss: 0.1734 - val_accuracy: 0.9400
Epoch 59/100
466/466 [==============================] - 0s 87us/step - loss: 0.0364 - accuracy: 0.9957 - val_loss: 0.2070 - val_accuracy: 0.9350
Epoch 60/100
466/466 [==============================] - 0s 84us/step - loss: 0.0374 - accuracy: 0.9936 - val_loss: 0.1956 - val_accuracy: 0.9350
Epoch 61/100
466/466 [==============================] - 0s 98us/step - loss: 0.0402 - accuracy: 0.9957 - val_loss: 0.2076 - val_accuracy: 0.9350
Epoch 62/100
466/466 [==============================] - 0s 105us/step - loss: 0.0324 - accuracy: 1.0000 - val_loss: 0.2309 - val_accuracy: 0.9250
Epoch 63/100
466/466 [==============================] - 0s 93us/step - loss: 0.0400 - accuracy: 0.9936 - val_loss: 0.1611 - val_

In [33]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

200/200 [==============================] - 0s 81us/step
over-sampling test accuracy: 95.00%


In [34]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [35]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 403us/step - loss: 6.3849 - accuracy: 0.4807 - val_loss: 3.6086 - val_accuracy: 0.7000
Epoch 2/100
466/466 [==============================] - 0s 113us/step - loss: 4.4832 - accuracy: 0.5987 - val_loss: 2.1237 - val_accuracy: 0.7300
Epoch 3/100
466/466 [==============================] - 0s 108us/step - loss: 3.3203 - accuracy: 0.6545 - val_loss: 1.5561 - val_accuracy: 0.7350
Epoch 4/100
466/466 [==============================] - 0s 118us/step - loss: 3.1477 - accuracy: 0.6695 - val_loss: 1.2666 - val_accuracy: 0.7800
Epoch 5/100
466/466 [==============================] - 0s 114us/step - loss: 3.1070 - accuracy: 0.6803 - val_loss: 1.1050 - val_accuracy: 0.8000
Epoch 6/100
466/466 [==============================] - 0s 111us/step - loss: 3.2744 - accuracy: 0.6674 - val_loss: 1.1982 - val_accuracy: 0.7950
Epoch 7/100
466/466 [==============================] - 0s 105us/step - loss: 3.5279 

Epoch 57/100
466/466 [==============================] - 0s 115us/step - loss: 1.6941 - accuracy: 0.7747 - val_loss: 0.9500 - val_accuracy: 0.8850
Epoch 58/100
466/466 [==============================] - 0s 93us/step - loss: 1.5860 - accuracy: 0.7554 - val_loss: 1.1093 - val_accuracy: 0.8950
Epoch 59/100
466/466 [==============================] - 0s 88us/step - loss: 1.5658 - accuracy: 0.7403 - val_loss: 0.9870 - val_accuracy: 0.9000
Epoch 60/100
466/466 [==============================] - 0s 101us/step - loss: 1.3839 - accuracy: 0.7918 - val_loss: 0.9699 - val_accuracy: 0.9100
Epoch 61/100
466/466 [==============================] - 0s 93us/step - loss: 1.2233 - accuracy: 0.8069 - val_loss: 1.0887 - val_accuracy: 0.8800
Epoch 62/100
466/466 [==============================] - 0s 92us/step - loss: 1.4271 - accuracy: 0.7532 - val_loss: 1.0500 - val_accuracy: 0.9100
Epoch 63/100
466/466 [==============================] - 0s 93us/step - loss: 1.3704 - accuracy: 0.7704 - val_loss: 1.0593 - val_

In [37]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

200/200 [==============================] - 0s 102us/step
over-sampling test accuracy: 93.50%


In [38]:
############## Random Forest ##############

In [39]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.00%


In [41]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.9787234  1.         0.94623656 0.95698925 0.94623656]
0.9656371539693435
